## Problem 3
Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find the 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries).

In [7]:
from xml.etree import ElementTree as ET
import pandas as pd

In [8]:
# Get data.
document_tree = ET.parse("data/mondial_database.xml",)
document_root = document_tree.getroot()

# Create a data frame. 
df_pop_info = pd.DataFrame(columns=["Country","City", "Year", "Population", "Ethnicity", "Percentage"])

# Use an iterator to get all of the information we need.
for country in document_root.getiterator("country"):
    country_name = country.find("name").text
    for egroup in country.findall('ethnicgroup'):
        if egroup !=None:
            percentage = float(egroup.attrib["percentage"])/100
            
        for city in country.iter("city"):
            city_name = city.find("name").text

            if city.findall("population") is None:
                continue;

            for pop in city.findall("population"):
                year = pop.attrib["year"]
                population = pop.text

                df_pop_info = df_pop_info.append(
                {
                    "Country": country_name,
                    "City": city_name,
                    "Year": year,
                    "Population": population,
                    "Ethnicity": egroup.text,
                    "Percentage": percentage
                }, ignore_index=True)
            
# Check the data.
df_pop_info.head()

,Country,City,Year,Population,Ethnicity,Percentage
0,Albania,Tirana,1987,192000,Albanian,0.95
1,Albania,Tirana,1990,244153,Albanian,0.95
2,Albania,Tirana,2011,418495,Albanian,0.95
3,Albania,Shkodër,1987,62000,Albanian,0.95
4,Albania,Shkodër,2011,77075,Albanian,0.95


In [10]:
# Make a copy of the data frame.
df_pop_info_c = df_pop_info.copy()

# Create a new data frame that will hold only the most recent data on populations.
df_latest_pop_info = pd.DataFrame(columns=["Country", "City", "Year", "Population", "Ethnicity", "Percentage"])

# Change the population column to numeric so that it can be used properly later.
df_pop_info_c["Population"] = pd.to_numeric(df_pop_info_c["Population"])

# Make sure that the cities are unique.
cities = df_pop_info["City"].unique()

# Populate the new data frame with only the most recent population measurement for each city.
for city in cities:
    city_pop = df_pop_info_c[df_pop_info_c.City == city]
    latest_measurement = city_pop[city_pop["Year"] == city_pop["Year"].max()]
    df_latest_pop_info = df_latest_pop_info.append(latest_measurement)

In [12]:
# Make a copy of the data frame.
df_latest_pop_info_c = df_latest_pop_info.copy()

# Create a new column to find the actual population of each ethnic group.
df_latest_pop_info_c["Adjusted_pop"] = df_latest_pop_info_c.Population * df_latest_pop_info_c.Percentage

# Delete unnecessary columns.
del df_latest_pop_info_c["Year"]
del df_latest_pop_info_c["City"]
del df_latest_pop_info_c["Country"]
del df_latest_pop_info_c["Population"]
del df_latest_pop_info_c["Percentage"]

# Sum up the populations by their ethnicity and then sort.
df_latest_pop_info_c = df_latest_pop_info_c.groupby("Ethnicity").sum()
df_latest_pop_info_c.sort_values('Adjusted_pop', ascending = False).head()

,Adjusted_pop
Ethnicity,
Han Chinese,2.942174e+08
European,1.689675e+08
Indo-Aryan,9.244288e+07
Russian,6.368289e+07
Mestizo,6.291310e+07
